In [1]:
#Installing the packages
!pip install --upgrade pip
!pip install datasets

In [2]:
from datasets import load_dataset

raw_dataset = load_dataset("glue", "mrpc")
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [3]:
# Signle dataset from the train split
raw_dataset["validation"][87]

{'sentence1': 'However , EPA officials would not confirm the 20 percent figure .',
 'sentence2': 'Only in the past few weeks have officials settled on the 20 percent figure .',
 'label': 0,
 'idx': 812}

In [4]:
# features of the datase
raw_dataset["test"].features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

### Tokenize

In [79]:
from huggingface_hub import login
login() # You will be prompted for your HF key, which will then be saved locally

In [5]:
from transformers import AutoTokenizer
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [6]:
tokenized_sentence_1 = tokenizer(raw_dataset["train"]["sentence1"])
tokenized_sentence_2 = tokenizer(raw_dataset["train"]["sentence2"])

In [7]:
input = tokenizer(raw_dataset["train"][15]["sentence1"])
input2 = tokenizer(raw_dataset["train"][15]["sentence2"])
print(input)
print(input2)

{'input_ids': [101, 24049, 2001, 2087, 3728, 3026, 3580, 2343, 2005, 1996, 9722, 1004, 4132, 9340, 12439, 2964, 2449, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [101, 3026, 3580, 2343, 4388, 24049, 1010, 3839, 2132, 1997, 1996, 9722, 1998, 4132, 9340, 12439, 2964, 3131, 1010, 2097, 2599, 1996, 2047, 9178, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [8]:
inputs = tokenizer(raw_dataset["train"][15]["sentence1"],raw_dataset["train"][15]["sentence2"])
print(inputs)

{'input_ids': [101, 24049, 2001, 2087, 3728, 3026, 3580, 2343, 2005, 1996, 9722, 1004, 4132, 9340, 12439, 2964, 2449, 1012, 102, 3026, 3580, 2343, 4388, 24049, 1010, 3839, 2132, 1997, 1996, 9722, 1998, 4132, 9340, 12439, 2964, 3131, 1010, 2097, 2599, 1996, 2047, 9178, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [9]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'rudder',
 'was',
 'most',
 'recently',
 'senior',
 'vice',
 'president',
 'for',
 'the',
 'developer',
 '&',
 'platform',
 'evan',
 '##gel',
 '##ism',
 'business',
 '.',
 '[SEP]',
 'senior',
 'vice',
 'president',
 'eric',
 'rudder',
 ',',
 'formerly',
 'head',
 'of',
 'the',
 'developer',
 'and',
 'platform',
 'evan',
 '##gel',
 '##ism',
 'unit',
 ',',
 'will',
 'lead',
 'the',
 'new',
 'entity',
 '.',
 '[SEP]']

In [10]:
# tokenize funtion for batch processing

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [11]:
tokenized_dataset =  raw_dataset.map(tokenize_function, batched=True)
tokenized_dataset["train"][4]

{'sentence1': 'The stock rose $ 2.11 , or about 11 percent , to close Friday at $ 21.51 on the New York Stock Exchange .',
 'sentence2': 'PG & E Corp. shares jumped $ 1.63 or 8 percent to $ 21.03 on the New York Stock Exchange on Friday .',
 'label': 1,
 'idx': 4,
 'input_ids': [101,
  1996,
  4518,
  3123,
  1002,
  1016,
  1012,
  2340,
  1010,
  2030,
  2055,
  2340,
  3867,
  1010,
  2000,
  2485,
  5958,
  2012,
  1002,
  2538,
  1012,
  4868,
  2006,
  1996,
  2047,
  2259,
  4518,
  3863,
  1012,
  102,
  18720,
  1004,
  1041,
  13058,
  1012,
  6661,
  5598,
  1002,
  1015,
  1012,
  6191,
  2030,
  1022,
  3867,
  2000,
  1002,
  2538,
  1012,
  6021,
  2006,
  1996,
  2047,
  2259,
  4518,
  3863,
  2006,
  5958,
  1012,
  102],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  

In [12]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Observe different input ids length after tokenization

In [13]:
samples = tokenized_dataset["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
print(samples)

ids_len = [len(ids) for ids in samples["input_ids"]]
print(ids_len)

{'label': [1, 0, 1, 0, 1, 1, 0, 1], 'input_ids': [[101, 2572, 3217, 5831, 5496, 2010, 2567, 1010, 3183, 2002, 2170, 1000, 1996, 7409, 1000, 1010, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 7727, 2000, 2032, 2004, 2069, 1000, 1996, 7409, 1000, 1010, 2572, 3217, 5831, 5496, 2010, 2567, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102], [101, 9805, 3540, 11514, 2050, 3079, 11282, 2243, 1005, 1055, 2077, 4855, 1996, 4677, 2000, 3647, 4576, 1999, 2687, 2005, 1002, 1016, 1012, 1019, 4551, 1012, 102, 9805, 3540, 11514, 2050, 4149, 11282, 2243, 1005, 1055, 1999, 2786, 2005, 1002, 6353, 2509, 2454, 1998, 2853, 2009, 2000, 3647, 4576, 2005, 1002, 1015, 1012, 1022, 4551, 1999, 2687, 1012, 102], [101, 2027, 2018, 2405, 2019, 15147, 2006, 1996, 4274, 2006, 2238, 2184, 1010, 5378, 1996, 6636, 2005, 5096, 1010, 2002, 2794, 1012, 102, 2006, 2238, 2184, 1010, 1996, 2911, 1005, 1055, 5608, 2018, 2405, 2019, 15147, 2006, 1996, 4274, 1010, 5378, 1996, 14792, 2005, 5096, 1012, 102], [101, 21

Now see how collator fix this variable length

In [14]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 67]),
 'token_type_ids': torch.Size([8, 67]),
 'attention_mask': torch.Size([8, 67]),
 'labels': torch.Size([8])}

### initializing the Trainer

In [15]:
from transformers import TrainingArguments
training_args = TrainingArguments("test-trainer")

### Load the checkpoint

In [16]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset= tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    # tokenizer=tokenizer,      #deprecated
    processing_class=tokenizer
)

In [18]:
trainer.train()

Step,Training Loss
500,0.565900
1000,0.339300


TrainOutput(global_step=1377, training_loss=0.38053408687151075, metrics={'train_runtime': 55.0265, 'train_samples_per_second': 199.976, 'train_steps_per_second': 25.024, 'total_flos': 405114969714960.0, 'train_loss': 0.38053408687151075, 'epoch': 3.0})

### Basic evaluation

In [19]:
predictions = trainer.predict(tokenized_dataset["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


In [20]:
predictions.predictions

array([[-2.7865925 ,  3.3046844 ],
       [ 2.5662029 , -2.8894274 ],
       [ 1.0556097 , -0.81592697],
       [-2.6189976 ,  3.0776405 ],
       [ 2.3464227 , -2.8323061 ],
       [-2.649227  ,  3.1177146 ],
       [-2.6901915 ,  3.1795273 ],
       [-2.6955612 ,  3.214234  ],
       [-2.673455  ,  3.1229036 ],
       [-2.6811433 ,  3.1904345 ],
       [-2.7087827 ,  3.2445984 ],
       [ 2.5615673 , -2.93583   ],
       [ 2.706623  , -2.9594216 ],
       [-2.7448473 ,  3.161352  ],
       [-2.8113616 ,  3.3501692 ],
       [ 0.8412137 , -0.7337403 ],
       [-2.8029342 ,  3.3453996 ],
       [ 0.25675583,  0.16048515],
       [-2.852763  ,  3.38552   ],
       [ 1.7327303 , -1.9107269 ],
       [ 2.3028345 , -2.7093072 ],
       [-2.5352528 ,  2.952558  ],
       [ 2.56475   , -2.9771361 ],
       [-2.844524  ,  3.3269508 ],
       [-2.773433  ,  3.2944746 ],
       [-1.7231929 ,  2.1131    ],
       [-2.0153131 ,  2.3776257 ],
       [-2.814791  ,  3.3407087 ],
       [-2.6625237 ,

In [21]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)
print(preds)

[1 0 0 1 0 1 1 1 1 1 1 0 0 1 1 0 1 0 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 0 1 1 0 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 1 0 1 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 0 1 0
 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 1 0 1 1 0 0 1 1 1 0 0 1 0 1 1 1
 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 0 0 0 1 1 1 1 0 1 1 1 1 1 1 1
 1 0 1 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1
 1 0 1 1 1 1 1 1 1 1 0 0 0 0 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 0 0 1 0 0 1 1 1
 1 1 1 1 1 1 0 1 0 0 1 0 1 1 1 1 1 1 0 1 0 1 1 0 0 0 1 1 1 1 1 0 0 1 1 1 0
 0 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 0 1 0 1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1 1 1 0 0
 1 1 1 1 0 0 0 0 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 0
 1]


In [27]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8602941176470589, 'f1': 0.901213171577123}

In [33]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [34]:
training_args = TrainingArguments("test-trainer", eval_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
    
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.580483,0.865196,0.901961
2,0.182000,0.792968,0.865196,0.905660
3,0.078100,0.810600,0.870098,0.909091


TrainOutput(global_step=1377, training_loss=0.10356380736905765, metrics={'train_runtime': 65.9627, 'train_samples_per_second': 166.822, 'train_steps_per_second': 20.875, 'total_flos': 405114969714960.0, 'train_loss': 0.10356380736905765, 'epoch': 3.0})